In [1]:
import pandas as pd
import os

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [2]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)
    # Your code that causes the warning


In [3]:
inp_filename = 'pi_pull_m6_2024-01-14.csv'
data_dir = 'data'

raw_data = pd.read_csv(os.path.join(data_dir, inp_filename))

/var/folders/mp/6wd7514s0s1gfpl_csdmpfh80000gn/T/ipykernel_1552/2924783605.py:4: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(os.path.join(data_dir, inp_filename))


In [43]:
import pandas as pd

def merge_data_with_time(df, window='1T', agg_func='mean', verbose=False):
    """
    Merge data columns with their corresponding time columns in a DataFrame.

    :param df: pandas DataFrame with data and time columns
    :param time_frame: time period for resampling (e.g., '1H' for 1 hour)
    :param agg_func: aggregation function (e.g., 'mean', 'sum', 'max')
    :return: pandas DataFrame after merging
    """
    # Initialize an empty list to store the series
    series_list = []

    # Determine the aggregation function
    if isinstance(agg_func, str):
        agg_func = getattr(pd.core.groupby.SeriesGroupBy, agg_func)

    # Identify and process each data column
    for column in df.columns:
        # Check if the column is a data column (not a time column)
        if not column.endswith('.TIME'):
            # Get the corresponding time column
            time_column = column + '.TIME'

            
            # Check if the time column exists
            if time_column in df.columns:
                #print(column, time_column)
                # Create a Series with the data column values and time column as index
                series = pd.Series(df[column].values, index=pd.to_datetime(df[time_column]), name=column)


                #else:
                #    # Group by index (time) and calculate the mean for duplicate times
                series = series.groupby(series.index).median()
                series = series.resample('5T').mean()
                if window:
                    series = series.rolling(window=window).mean()
                if verbose:
                    print(series.head())
                
                series_list.append(series)
    try:
        # Merge all series with an outer join
        merged_df = pd.concat(series_list, axis=1, join='outer')
    except:
        return series_list

    return merged_df

# Example usage:
# Assume 'dataframe' is your existing DataFrame
# For default behavior
# merged_dataframe = merge_data_with_time(dataframe)

# For resampling every hour and summing values
# merged_dataframe = merge_data_with_time(dataframe, time_frame='1H', agg_func='sum')


In [44]:
%%time

#inp_filename = 'EX1-6-month.csv'
data_dir = 'data'

field_mappings = {}
#time_
ex_data = {}
processed_data = {}

for extruder in ['EX1', 'EX2', 'EX3', 'EX4']:
    print(extruder)
    ex_name_mapping = pd.read_csv(os.path.join(data_dir, f'{extruder}-6-month.csv'), 
                               delimiter='\t',
                               encoding='utf-16',
                               nrows=1)

    field_mappings[extruder] = dict(zip(ex_name_mapping.iloc[0], ex_name_mapping.columns ))

    #print(extruder)
    #print(name_mappings[extruder])


   # for field_key in name_mappings[extruder]:

    col_mapping = {key : key.split(' (')[0] for key, value in field_mappings[extruder].items() if key != 'Time'}

    field_mappings[extruder + '_column_map'] = col_mapping
    
    raw_fieldnames = sorted(col_mapping.values())
    time_keys = [fieldname + '.TIME' for fieldname in raw_fieldnames]
        
    #print(keys_filtered)
    #print(time_keys)

    keys_of_interest = sorted(raw_fieldnames + time_keys)

    ex_data[extruder] = raw_data[keys_of_interest]

    processed_df = merge_data_with_time(ex_data[extruder], verbose=False, window=None)

    
    
    processed_data[extruder] = processed_df
    
    for tk in time_keys:
        ex_data[extruder][tk] = pd.to_datetime(ex_data[extruder][tk])

        field_key = tk.replace('.TIME', '')

    print()
    

    #ex_data[extruder] = raw_data[keys_filtered]




EX1


<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 


EX2


<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 


EX3


<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 


EX4


<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 


CPU times: user 1min 23s, sys: 33.7 s, total: 1min 57s
Wall time: 2min 27s


<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
for ex in processed_data.keys():
    processed_df = processed_data[ex]
    print(ex)
    print(processed_df.isnull().sum()/len(processed_df))
    #print(processed_data[ex].head(1))
    print()

EX1
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV      0.010703
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV      0.275331
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT    0.999638
PT_5347.PV                                                   0.009959
SIC_5345.OUT                                                 0.999695
ST_5347.PV                                                   0.985997
TT_5347.PV                                                   0.863613
WT_5347.PV                                                   0.011466
dtype: float64

EX2
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5350.PV      0.114238
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5410.PV      0.687506
PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5410.OUT    0.999657
PT_5352.PV                                                   0.059694
SIC_5350.OUT                                                 0.999447
ST_5352.PV                                                   0.971

In [46]:
processed_data['EX1'].head(20)

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PT_5347.PV,SIC_5345.OUT,ST_5347.PV,TT_5347.PV,WT_5347.PV
2023-07-16 04:50:00,4.117500,0.969687,NaN,4057.106586,NaN,NaN,NaN,48.796735
2023-07-16 04:55:00,4.977000,0.976985,NaN,3996.547541,NaN,NaN,NaN,48.589401
2023-07-16 05:00:00,3.830000,0.971379,NaN,3998.819069,NaN,NaN,NaN,48.614656
2023-07-16 05:05:00,3.690000,0.971888,NaN,4076.056384,NaN,NaN,NaN,49.006142
2023-07-16 05:10:00,4.506000,0.970986,NaN,4133.956836,NaN,NaN,NaN,49.074539
2023-07-16 05:15:00,4.583333,0.968239,NaN,4139.360895,NaN,NaN,NaN,49.153383
2023-07-16 05:20:00,3.927500,0.966815,NaN,4166.419987,NaN,NaN,334.851562,49.221750
2023-07-16 05:25:00,4.960000,0.968451,NaN,4170.090208,NaN,NaN,NaN,49.255214
2023-07-16 05:30:00,4.833750,0.967986,NaN,4177.969916,NaN,NaN,NaN,49.217356
2023-07-16 05:35:00,4.707000,0.973103,NaN,4178.244786,NaN,NaN,NaN,49.253638


In [47]:
processed_data['EX2'].head(20)

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5350.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5410.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5410.OUT,PT_5352.PV,SIC_5350.OUT,ST_5352.PV,TT_5352.PV,WT_5352.PV
2023-07-16 04:50:00,5.200000,0.980000,NaN,3221.500208,NaN,NaN,NaN,60.076366
2023-07-16 04:55:00,3.350000,0.980000,NaN,3147.111020,NaN,NaN,272.890625,59.512808
2023-07-16 05:00:00,4.710000,0.995000,NaN,3217.413433,NaN,NaN,274.255859,59.979941
2023-07-16 05:05:00,3.550000,0.984286,NaN,3310.032572,NaN,NaN,NaN,60.364799
2023-07-16 05:10:00,2.345000,0.975000,NaN,3292.845131,NaN,NaN,NaN,60.667564
2023-07-16 05:15:00,3.466667,0.976667,NaN,3284.698269,NaN,NaN,276.605469,60.605415
2023-07-16 05:20:00,3.206667,NaN,NaN,3311.250962,NaN,NaN,NaN,60.559718
2023-07-16 05:25:00,4.720000,0.975000,NaN,3287.348437,NaN,NaN,274.621094,60.629223
2023-07-16 05:30:00,3.607500,0.980000,NaN,3283.322938,NaN,27.813858,275.726562,60.445798
2023-07-16 05:35:00,2.345000,NaN,NaN,3291.354810,NaN,NaN,NaN,60.575610


In [48]:
processed_data['EX3'].iloc[20:40]

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5355.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5415.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5415.OUT,PT_5357.PV,SIC_5355.OUT,ST_5357.PV,TT_5357.PV,WT_5357.PV
2023-07-17 17:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 17:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-17 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
processed_data['EX4'].head(20)

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5360.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5420.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5420.OUT,PT_5362.PV,SIC_5360.OUT,ST_5362.PV,TT_5362.PV,WT_5362.PV
2023-07-16 04:50:00,3.410000,1.070000,NaN,1801.763074,NaN,NaN,NaN,37.122286
2023-07-16 04:55:00,4.675000,1.070756,NaN,1792.386690,NaN,NaN,NaN,37.019737
2023-07-16 05:00:00,4.131667,1.067364,NaN,1805.653592,NaN,NaN,8.323669,37.135512
2023-07-16 05:05:00,5.272500,1.069709,NaN,1787.643932,NaN,NaN,NaN,36.980019
2023-07-16 05:10:00,3.772500,1.071493,NaN,1789.340884,NaN,NaN,NaN,37.037894
2023-07-16 05:15:00,3.663333,1.072422,NaN,1798.641825,NaN,NaN,NaN,37.161429
2023-07-16 05:20:00,3.645000,1.071949,NaN,1812.039048,NaN,NaN,NaN,37.146233
2023-07-16 05:25:00,4.250000,1.068651,NaN,1811.566819,NaN,NaN,NaN,37.252147
2023-07-16 05:30:00,3.865000,1.072182,NaN,1812.149493,NaN,NaN,NaN,37.233721
2023-07-16 05:35:00,5.353333,1.076410,NaN,1814.321597,NaN,NaN,NaN,37.327504


In [73]:
test = processed_df[processed_df['ST_5347.PV'].notnull()]
test.sort_index().index[:50]

DatetimeIndex(['2023-07-16 09:19:00', '2023-07-16 17:09:00',
               '2023-07-17 01:00:00', '2023-07-17 08:50:00',
               '2023-07-17 16:05:00', '2023-07-17 16:06:00',
               '2023-07-17 16:07:00', '2023-07-17 23:58:00',
               '2023-07-18 07:48:00', '2023-07-18 15:38:00',
               '2023-07-18 23:29:00', '2023-07-19 07:19:00',
               '2023-07-19 15:10:00', '2023-07-19 23:00:00',
               '2023-07-20 06:50:00', '2023-07-20 11:09:00',
               '2023-07-20 12:29:00', '2023-07-20 12:30:00',
               '2023-07-20 20:20:00', '2023-07-21 04:10:00',
               '2023-07-21 12:01:00', '2023-07-21 19:51:00',
               '2023-07-22 03:41:00', '2023-07-22 11:32:00',
               '2023-07-22 19:22:00', '2023-07-23 03:13:00',
               '2023-07-23 11:03:00', '2023-07-23 18:53:00',
               '2023-07-24 02:44:00', '2023-07-24 10:34:00',
               '2023-07-24 18:25:00', '2023-07-25 02:15:00',
               '2023-07-

In [75]:
test.head(5)

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PT_5347.PV,SIC_5345.OUT,ST_5347.PV,TT_5347.PV,WT_5347.PV
2023-07-16 09:19:00,2.7350,0.970741,NaN,4067.157785,NaN,27.800705,NaN,48.920944
2023-07-16 17:09:00,2.7550,0.947826,NaN,4107.848800,NaN,27.796013,NaN,50.515979
2023-07-17 01:00:00,NaN,0.985714,NaN,4229.140625,NaN,27.820351,NaN,51.048469
2023-07-17 08:50:00,5.7975,0.967391,NaN,4159.743207,NaN,27.828751,NaN,50.613756
2023-07-17 16:05:00,NaN,0.946667,NaN,4411.497389,NaN,27.822044,350.408203,50.505443


In [30]:
field_mappings

{'EX1': {'Time': 'Date and Time',
  'PT_5347.PV (psi)': 'Extruder Pressure',
  'ST_5347.PV (RPM)': 'Screw Speed Output',
  'SIC_5345.OUT (%)': 'Screw Speed',
  'TT_5347.PV (°F)': 'Extruder Die Temp',
  'WT_5347.PV (%)': 'Extruder Thrust ',
  'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV (Amps)': 'Feed Screw Current (Amps)',
  'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV (Amps)': 'Discharge Conveyor Current (Amps)',
  'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT (%)': 'Discharge Conveyor Speed (%)'},
 'EX1_column_map': {'PT_5347.PV (psi)': 'PT_5347.PV',
  'ST_5347.PV (RPM)': 'ST_5347.PV',
  'SIC_5345.OUT (%)': 'SIC_5345.OUT',
  'TT_5347.PV (°F)': 'TT_5347.PV',
  'WT_5347.PV (%)': 'WT_5347.PV',
  'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV (Amps)': 'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV',
  'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV (Amps)': 'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV',
  'PJ00

In [29]:
ex_data['EX1'].head()

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT.TIME,PT_5347.PV,PT_5347.PV.TIME,SIC_5345.OUT,SIC_5345.OUT.TIME,ST_5347.PV,ST_5347.PV.TIME,TT_5347.PV,TT_5347.PV.TIME,WT_5347.PV,WT_5347.PV.TIME
0,5.56,2023-07-16 04:51:02,0.94,2023-07-16 04:50:52,6.952558e-310,2023-07-17 16:06:18,4070.291016,2023-07-16 04:50:50,6.952558e-310,2023-07-17 16:06:15,27.800705,2023-07-16 09:19:27,334.851562,2023-07-16 05:23:22,49.097092,2023-07-16 04:50:53
1,0.00,2023-07-16 04:51:03,0.98,2023-07-16 04:50:53,7.000000e+01,2023-07-17 16:07:48,3941.896484,2023-07-16 04:50:51,3.500000e+01,2023-07-17 16:07:46,27.796013,2023-07-16 17:09:49,335.320312,2023-07-16 06:17:56,48.470383,2023-07-16 04:50:56
2,5.34,2023-07-16 04:53:45,0.96,2023-07-16 04:50:55,8.500000e+01,2023-08-03 09:32:37,4167.226562,2023-07-16 04:50:52,2.500000e+01,2023-07-25 14:28:38,27.820351,2023-07-17 01:00:20,337.341797,2023-07-16 07:31:15,48.842682,2023-07-16 04:51:09
3,5.57,2023-07-16 04:53:47,1.02,2023-07-16 04:50:56,1.000000e+02,2023-08-03 09:51:02,3930.353516,2023-07-16 04:50:53,NaN,NaT,27.828751,2023-07-17 08:50:49,341.666016,2023-07-16 10:29:42,48.621353,2023-07-16 04:51:22
4,5.51,2023-07-16 04:55:14,1.01,2023-07-16 04:50:57,NaN,NaT,4192.916016,2023-07-16 04:50:54,NaN,NaT,27.822044,2023-07-17 16:05:54,344.843750,2023-07-16 11:05:26,48.816040,2023-07-16 04:51:35


In [ ]:
ex_data['EX1'].head()

In [ ]:
ex_data['EX1'].info()

In [22]:
ex_data['EX2'].head()

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT.TIME,PT_5347.PV,PT_5347.PV.TIME,SIC_5345.OUT,SIC_5345.OUT.TIME,ST_5347.PV,ST_5347.PV.TIME,TT_5347.PV,TT_5347.PV.TIME,WT_5347.PV,WT_5347.PV.TIME
0,5.56,2023-07-16 04:51:02,0.94,2023-07-16 04:50:52,6.952558e-310,2023-07-17 16:06:18,4070.291016,2023-07-16 04:50:50,6.952558e-310,2023-07-17 16:06:15,27.800705,2023-07-16 09:19:27,334.851562,2023-07-16 05:23:22,49.097092,2023-07-16 04:50:53
1,0.00,2023-07-16 04:51:03,0.98,2023-07-16 04:50:53,7.000000e+01,2023-07-17 16:07:48,3941.896484,2023-07-16 04:50:51,3.500000e+01,2023-07-17 16:07:46,27.796013,2023-07-16 17:09:49,335.320312,2023-07-16 06:17:56,48.470383,2023-07-16 04:50:56
2,5.34,2023-07-16 04:53:45,0.96,2023-07-16 04:50:55,8.500000e+01,2023-08-03 09:32:37,4167.226562,2023-07-16 04:50:52,2.500000e+01,2023-07-25 14:28:38,27.820351,2023-07-17 01:00:20,337.341797,2023-07-16 07:31:15,48.842682,2023-07-16 04:51:09
3,5.57,2023-07-16 04:53:47,1.02,2023-07-16 04:50:56,1.000000e+02,2023-08-03 09:51:02,3930.353516,2023-07-16 04:50:53,NaN,NaN,27.828751,2023-07-17 08:50:49,341.666016,2023-07-16 10:29:42,48.621353,2023-07-16 04:51:22
4,5.51,2023-07-16 04:55:14,1.01,2023-07-16 04:50:57,NaN,NaN,4192.916016,2023-07-16 04:50:54,NaN,NaN,27.822044,2023-07-17 16:05:54,344.843750,2023-07-16 11:05:26,48.816040,2023-07-16 04:51:35


In [23]:
ex_data['EX3'].head()

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT.TIME,PT_5347.PV,PT_5347.PV.TIME,SIC_5345.OUT,SIC_5345.OUT.TIME,ST_5347.PV,ST_5347.PV.TIME,TT_5347.PV,TT_5347.PV.TIME,WT_5347.PV,WT_5347.PV.TIME
0,5.56,2023-07-16 04:51:02,0.94,2023-07-16 04:50:52,6.952558e-310,2023-07-17 16:06:18,4070.291016,2023-07-16 04:50:50,6.952558e-310,2023-07-17 16:06:15,27.800705,2023-07-16 09:19:27,334.851562,2023-07-16 05:23:22,49.097092,2023-07-16 04:50:53
1,0.00,2023-07-16 04:51:03,0.98,2023-07-16 04:50:53,7.000000e+01,2023-07-17 16:07:48,3941.896484,2023-07-16 04:50:51,3.500000e+01,2023-07-17 16:07:46,27.796013,2023-07-16 17:09:49,335.320312,2023-07-16 06:17:56,48.470383,2023-07-16 04:50:56
2,5.34,2023-07-16 04:53:45,0.96,2023-07-16 04:50:55,8.500000e+01,2023-08-03 09:32:37,4167.226562,2023-07-16 04:50:52,2.500000e+01,2023-07-25 14:28:38,27.820351,2023-07-17 01:00:20,337.341797,2023-07-16 07:31:15,48.842682,2023-07-16 04:51:09
3,5.57,2023-07-16 04:53:47,1.02,2023-07-16 04:50:56,1.000000e+02,2023-08-03 09:51:02,3930.353516,2023-07-16 04:50:53,NaN,NaN,27.828751,2023-07-17 08:50:49,341.666016,2023-07-16 10:29:42,48.621353,2023-07-16 04:51:22
4,5.51,2023-07-16 04:55:14,1.01,2023-07-16 04:50:57,NaN,NaN,4192.916016,2023-07-16 04:50:54,NaN,NaN,27.822044,2023-07-17 16:05:54,344.843750,2023-07-16 11:05:26,48.816040,2023-07-16 04:51:35


In [20]:
ex_data['EX4'].head()

,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV.TIME,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT,PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT.TIME,PT_5347.PV,PT_5347.PV.TIME,SIC_5345.OUT,SIC_5345.OUT.TIME,ST_5347.PV,ST_5347.PV.TIME,TT_5347.PV,TT_5347.PV.TIME,WT_5347.PV,WT_5347.PV.TIME
0,5.56,2023-07-16 04:51:02,0.94,2023-07-16 04:50:52,6.952558e-310,2023-07-17 16:06:18,4070.291016,2023-07-16 04:50:50,6.952558e-310,2023-07-17 16:06:15,27.800705,2023-07-16 09:19:27,334.851562,2023-07-16 05:23:22,49.097092,2023-07-16 04:50:53
1,0.00,2023-07-16 04:51:03,0.98,2023-07-16 04:50:53,7.000000e+01,2023-07-17 16:07:48,3941.896484,2023-07-16 04:50:51,3.500000e+01,2023-07-17 16:07:46,27.796013,2023-07-16 17:09:49,335.320312,2023-07-16 06:17:56,48.470383,2023-07-16 04:50:56
2,5.34,2023-07-16 04:53:45,0.96,2023-07-16 04:50:55,8.500000e+01,2023-08-03 09:32:37,4167.226562,2023-07-16 04:50:52,2.500000e+01,2023-07-25 14:28:38,27.820351,2023-07-17 01:00:20,337.341797,2023-07-16 07:31:15,48.842682,2023-07-16 04:51:09
3,5.57,2023-07-16 04:53:47,1.02,2023-07-16 04:50:56,1.000000e+02,2023-08-03 09:51:02,3930.353516,2023-07-16 04:50:53,NaN,NaN,27.828751,2023-07-17 08:50:49,341.666016,2023-07-16 10:29:42,48.621353,2023-07-16 04:51:22
4,5.51,2023-07-16 04:55:14,1.01,2023-07-16 04:50:57,NaN,NaN,4192.916016,2023-07-16 04:50:54,NaN,NaN,27.822044,2023-07-17 16:05:54,344.843750,2023-07-16 11:05:26,48.816040,2023-07-16 04:51:35


In [17]:
ex1_field_mapping = 

In [18]:
ex1_field_mapping

{'Time': 'Date and Time',
 'PT_5347.PV (psi)': 'Extruder Pressure',
 'ST_5347.PV (RPM)': 'Screw Speed Output',
 'SIC_5345.OUT (%)': 'Screw Speed',
 'TT_5347.PV (°F)': 'Extruder Die Temp',
 'WT_5347.PV (%)': 'Extruder Thrust ',
 'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5345.PV (Amps)': 'Feed Screw Current (Amps)',
 'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.IT_5405.PV (Amps)': 'Discharge Conveyor Current (Amps)',
 'PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.SIC_5405.OUT (%)': 'Discharge Conveyor Speed (%)'}

In [1]:
ex1_raw_data.head()

NameError: name 'ex1_raw_data' is not defined

In [17]:
ex1_code_names = ex1_raw_data.iloc[0]
ex1_raw_data = ex1_raw_data.iloc[1:]

#test = ex1_raw_data.infer_objects()

#for types in zip(ex1_raw_data.dtypes, test.dtypes):
#    print(types)
ex1_raw_data.dtypes



Date and Time                        object
Extruder Pressure                    object
Screw Speed Output                   object
Screw Speed                          object
Extruder Die Temp                    object
Extruder Thrust                      object
Feed Screw Current (Amps)            object
Discharge Conveyor Current (Amps)    object
Discharge Conveyor Speed (%)         object
dtype: object

In [19]:
ex1_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396439 entries, 3 to 396441
Data columns (total 9 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Date and Time                      396439 non-null  object
 1   Extruder Pressure                  396439 non-null  object
 2   Screw Speed Output                 396439 non-null  object
 3   Screw Speed                        396439 non-null  object
 4   Extruder Die Temp                  396439 non-null  object
 5   Extruder Thrust                    396439 non-null  object
 6   Feed Screw Current (Amps)          396439 non-null  object
 7   Discharge Conveyor Current (Amps)  396439 non-null  object
 8   Discharge Conveyor Speed (%)       396439 non-null  object
dtypes: object(9)
memory usage: 27.2+ MB


In [21]:
has_null_vals = ex1_raw_data.apply(lambda row: '<null>' in row.values, axis=1)

In [14]:
ex1_raw_data.iloc[0]

Date and Time                                                                     Time
Extruder Pressure                                                     PT_5347.PV (psi)
Screw Speed Output                                                    ST_5347.PV (RPM)
Screw Speed                                                           SIC_5345.OUT (%)
Extruder Die Temp                                                      TT_5347.PV (°F)
Extruder Thrust                                                         WT_5347.PV (%)
Feed Screw Current (Amps)            PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.I...
Discharge Conveyor Current (Amps)    PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.I...
Discharge Conveyor Speed (%)         PJ001959_EWS:RSLinx Enterprise:PJ001992_SITE.S...
Name: 0, dtype: object

In [22]:
test = ex1_raw_data[has_null_vals]

In [27]:
len(t)

28

In [25]:
for t in test.columns:
    print(test[t].value_counts())

Date and Time
3/5/2023 12:24 AM     11
4/17/2023 9:40 AM     11
3/29/2023 1:42 PM     10
4/20/2023 9:55 AM      5
2/26/2023 2:53 PM      4
4/17/2023 9:59 AM      4
4/17/2023 10:42 AM     4
4/20/2023 9:38 AM      4
2/26/2023 2:52 PM      3
3/29/2023 1:43 PM      1
Name: count, dtype: int64
Extruder Pressure
<null>      39
-507.5      10
4,214.55     4
6,928.36     1
5,486.19     1
6,128.92     1
4,413.36     1
Name: count, dtype: int64
Screw Speed Output
<null>    23
-1.16      9
28.81      7
27.78      6
25.8       5
28.79      3
27.82      1
28.8       1
28.78      1
25.81      1
Name: count, dtype: int64
Screw Speed
<null>    43
25        10
35         4
Name: count, dtype: int64
Extruder Die Temp
<null>    26
-391.5     8
350.81     6
370.98     5
359.54     4
371.11     2
348.13     2
359.83     2
350.76     1
348.11     1
Name: count, dtype: int64
Extruder Thrust 
<null>    30
-3.63      9
58.55      5
70.3       4
58.39      2
44.05      2
71.38      2
74.69      1
75.94      1
4